In [79]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np
import re


In [51]:
driver = webdriver.Chrome("/Applications/chromedriver")


In [188]:
store_url = 'https://commande.bio-c-bon.fr/'
# need to manually select Paris 15 to get the biggest store
driver.get(store_url)



In [23]:
main_categories = []

main_category_list = driver.find_elements_by_class_name('category-subcategory-box')

for main_category in main_category_list:
    name = main_category.find_element_by_class_name('category-subcategory-name-text').text
    url = main_category.find_element_by_class_name('category-subcategory-name-text').get_attribute("href")
    main_categories.append({"name":name, 
                          "url": url
                          })
len(main_categories)

selected_categories = main_categories[:-4]

12

In [27]:
categories = []

for main_cat in selected_categories:
    parent_shelter = main_cat['name']
    driver.get(main_cat['url'])
    sleep(1)

    sub_category_list = driver.find_elements_by_class_name('category-subcategory-box')

    for main_category in sub_category_list:
        name = main_category.find_element_by_class_name('category-subcategory-name-text').text
        url = main_category.find_element_by_class_name('category-subcategory-name-text').get_attribute("href")
        categories.append({"parent_shelter": parent_shelter, 
                           "child_shelter": name, 
                            "url": url
                          })
    


len(categories)


51

In [189]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}
product_pages = []

for category in categories:
    url = category['url']
    driver.get(url)
    sleep(1)

    for i in range(3):
        actions = ActionChains(driver)
        try:
            driver.find_element_by_class_name("button-more-products").click()
        except:
            pass


    soup = BeautifulSoup(driver.page_source, 'lxml')
    try:
        results = soup.select('.product-grid')[0]
    except:
        results = ''

    product_pages.append({"shelter_parent": category['parent_shelter'],
                          "child_shelter": category['child_shelter'],
                          "url": url, 
                          "soup": results
                          })


        
len(product_pages)  




51

In [194]:
products = []
bad_products = []

for page in product_pages:
    shelter_parent = page['shelter_parent']
    shelter_child = page['child_shelter'] 
    
    if page['soup']:
    
        for product in page['soup'].find_all("li", class_="product-grid-item"):
            product_name = product.find("span", class_="desc").text
            product_link = product.find("a", class_="product-link")['href']
            product_image = product.find('img', class_="product-img")['src']
            store_name = "Bio c' Bon"
            regex = re.findall(r"[^/]+$", product_image)[0]
            product_ean = re.search(r"(\d+)", regex).group(1)
            product_price = product.find("div", class_="price-box").find("span", class_="price").text.replace('\xa0', '').replace(',', '.')
            try:
                product_price_per_unit = product.find("span", class_="piece").text.replace('\xa0', ' ').replace(',', '.')
            except:
                product_price_per_unit = product_price + '/ pièce'
                print(product_name)
            brand = ''


            products.append({"shelter_parent": shelter_parent,
                             "shelter_child": shelter_child,
                             "brand": brand,  
                             "description1": product_name,                             
                             "description": product_name,
                             "price": product_price[:-1],
                             "price_per_unit": product_price_per_unit,
                             #"quantity": quantity,
                             #"promo_price": promo_price,
                             "store": store_name,
                             "image_url": product_image,                    
                             "url": product_link,
                             "ean": product_ean
                            })
print(len(products))
print(len(bad_products))

Chou-fleur bio (à la pièce)
Persil plat bio, botte (à la pièce)
Salade Laitue bio (à la pièce)
Avocat bio (à la pièce)
Concombre bio (à la pièce)
Salade Batavia verte bio (à la pièce)
Chou blanc bio (à la pièce)
Panier de Fruits et Légumes bio - Spécial Bébé (à la pièce)
Panier de Fruits bio (à la pièce)
Panier de Légumes bio (à la pièce)
Mangue bio (à la pièce)
Noix de Coco bio (à la pièce)
4 gros œufs frais bio (à la pièce)
Boite 6 oeufs gros (à la pièce)
Boite 15 oeufs (à la pièce)
Boite 10 oeufs moyen 53/63 (à la pièce)
Hareng fumé doux MSC 200g (à la pièce)
899
0


In [200]:
product_data = pd.DataFrame(products)

# drop product without ean
print(len(product_data[product_data['ean'] == '41']))
product_data['ean'].replace('41', np.nan, inplace=True)
product_data = product_data.dropna(subset=['ean'])
print(len(product_data[product_data['ean'] == '41']))

19
0


In [201]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = product_data[product_data.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [shelter_parent, shelter_child, brand, description1, description, price, price_per_unit, store, image_url, url, ean]
Index: []


In [202]:
product_data = product_data.drop_duplicates()

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_biocbon.csv', index = None, header=True)
